# 04 - Preset Tuning for GPU Budget

This notebook explains how to tune memory/speed knobs without changing math semantics.

Preset intent:
- `gpu_min`: memory-first (safe default)
- `gpu_full`: accuracy/performance-first when GPU budget allows


In [1]:
from pathlib import Path
import sys

root = Path.cwd()
if not (root / "src").exists():
    root = root.parent
sys.path.insert(0, str(root))

from src_tensor.api import resolve_preset, compile_expval_program
from src.pauli_surrogate_python import PauliRotation, CliffordGate, PauliSum

In [2]:
base = resolve_preset("gpu_min")
print("base:", base)

custom = resolve_preset(
    "gpu_min",
    overrides={
        "max_weight": 6,
        "offload_keep": 1,
        "dtype": "float32",
    },
)
print("custom:", custom)

base: TensorSurrogatePreset(build_device='cuda', step_device='cpu', stream_device='cuda', dtype='float32', max_weight=5, max_xy=1000000000, offload_steps=True, offload_keep=1, offload_back=True)
custom: TensorSurrogatePreset(build_device='cuda', step_device='cpu', stream_device='cuda', dtype='float32', max_weight=6, max_xy=1000000000, offload_steps=True, offload_keep=1, offload_back=True)


## Example compile with targeted overrides

Common knobs:
- `max_weight`, `max_xy`: structural truncation strength
- `offload_steps`: whether to keep old sparse steps on CPU
- `step_device`, `stream_device`: storage vs compute devices


In [3]:
n_qubits = 2
circuit = [PauliRotation("ZZ", [0, 1], param_idx=0)]
obs = PauliSum(n_qubits)
obs.add_from_str("ZZ", 1.0, qubits=[0, 1])

program = compile_expval_program(
    circuit=circuit,
    observables=[obs],
    preset="gpu_min",
    preset_overrides={
        "max_weight": 6,
        "offload_keep": 1,
    },
)

print("program preset:", program.preset)

propagate: 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]

program preset: TensorSurrogatePreset(build_device='cuda', step_device='cpu', stream_device='cuda', dtype='float32', max_weight=6, max_xy=1000000000, offload_steps=True, offload_keep=1, offload_back=True)


## Practical policy
1. Start from `gpu_min` defaults.
2. If memory is tight, reduce `max_weight` first.
3. If speed is too low and memory allows, reduce offloading / move step storage closer to compute device.
4. Always re-check error vs exact small-n baseline after tuning.
